In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

# Function to load images from a directory, resizing them to a specific target size
def load_images_from_directory(directory, target_size=(224, 224)):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Check for image file types
            img = Image.open(os.path.join(directory, filename))
            img = img.resize(target_size)  # Resize image to the target size (224x224)
            img_array = np.array(img)  # Convert the image to a numpy array
            images.append(img_array)  # Add to the image list
    return np.array(images)

# Assuming the dataset is already loaded and preprocessed (replace 'load_labels' with your method)
X = load_images_from_directory('/content/atec')  # Load image data from directory
y = load_labels('/content/LRE_1280_0780570508_160ECM_N0600000SCAM01280_0010I6J01.png')  # Load corresponding labels

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize image data (scaling pixel values to the range [0, 1])
X_train = X_train / 255.0
X_test = X_test / 255.0

# Binarizing labels: 1 if percentage is > 60%, else 0
y_train_binary = np.where(y_train > 60, 1, 0)

# Data augmentation to artificially increase the diversity of the training set
datagen = ImageDataGenerator(
    rotation_range=20,  # Random rotations
    width_shift_range=0.2,  # Random horizontal shifts
    height_shift_range=0.2,  # Random vertical shifts
    shear_range=0.15,  # Shearing
    zoom_range=0.15,  # Zooming
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill in newly created pixels
)

# Fit the data generator to the training data
datagen.fit(X_train)

# Function to build the CNN model
def build_model():
    model = Sequential([  # Sequential model allows stacking layers
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),  # Convolutional layer with 32 filters
        MaxPooling2D(pool_size=(2, 2)),  # Max pooling to down-sample

        Conv2D(64, (3, 3), activation='relu'),  # Second convolutional layer with 64 filters
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(128, (3, 3), activation='relu'),  # Third convolutional layer with 128 filters
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),  # Flatten the matrix to a vector for the dense layer
        Dense(128, activation='relu'),  # Fully connected layer with 128 units
        Dropout(0.5),  # Dropout to prevent overfitting
        Dense(1, activation='sigmoid')  # Final binary classification layer (exploitable or not)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001),  # Adam optimizer with a learning rate of 0.001
                  loss='binary_crossentropy',  # Binary cross-entropy loss function
                  metrics=['accuracy'])  # Metrics to monitor: accuracy

    return model

# Instantiate and build the model
model = build_model()

# Set batch size and number of epochs for training
batch_size = 32
history = model.fit(datagen.flow(X_train, y_train_binary, batch_size=batch_size),  # Train using augmented data
                    validation_data=(X_test, y_test_binary),  # Validate on the test data
                    steps_per_epoch=len(X_train) // batch_size,
                    epochs=20)  # Train for 20 epochs

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test_binary, verbose=2)
print(f'\nTest Accuracy: {test_acc * 100:.2f}%')

# Binary classification: Labels for resources over 60%
data['label'] = np.where(data['resource_percentage'] > 60, 1, 0)
model.add(Dense(1, activation='sigmoid'))  # Adding the binary classification layer
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Plot training and validation accuracy over epochs
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/content/atec'

FileNotFoundError: [Errno 2] No such file or directory: '/content/atec'